<a href="https://colab.research.google.com/github/tlacaelel666/projectalpha1/blob/main/estadusQuantum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Representación Computacional de un Estado Cuántico Básico

Este repositorio contiene un ejemplo sencillo de cómo representar un estado cuántico básico (un qubit) utilizando Python y la librería NumPy. El objetivo es ilustrar la conexión matemática entre los estados base cuánticos $|0⟩$ y $|1⟩$ y su representación como vectores en un entorno de programación clásico.

## Conceptos Clave

*   **Qubit:** La unidad básica de información cuántica. A diferencia de un bit clásico que puede ser 0 o 1, un qubit puede estar en un estado de superposición de $|0⟩$ y $|1⟩$.
*   **Estados Base:** Los estados fundamentales de un qubit, denotados como $|0⟩$ y $|1⟩$. Matemáticamente, se representan como vectores ortonormales. En este ejemplo, utilizamos la representación vectorial estándar:
    *   $|0⟩ = \begin{pmatrix} 1 \\ 0 \end{pmatrix}$
    *   $|1⟩ = \begin{pmatrix} 0 \\ 1 \end{pmatrix}$
*   **Estado Cuántico General:** Un estado cuántico arbitrario de un qubit puede expresarse como una combinación lineal de los estados base:
    $|\psi⟩ = \alpha|0⟩ + \beta|1⟩$
    donde $\alpha$ y $\beta$ son números complejos llamados amplitudes de probabilidad. La suma de los cuadrados de las magnitudes de las amplitudes debe ser igual a 1 ($|\alpha|^2 + |\beta|^2 = 1$) para asegurar que el estado esté normalizado.
*   **Representación Computacional:** Utilizar herramientas de programación clásica (como NumPy en Python) para modelar y manipular las representaciones matemáticas de los estados cuánticos. Esto es fundamental para la simulación de sistemas cuánticos.

## Código

El código en este repositorio realiza los siguientes pasos:

1.  Define variables numéricas (`x`, `y`, `z`, `n`, `w`) que se utilizan para calcular las amplitudes.
2.  Calcula las amplitudes `c0` y `c1` (correspondientes a $\alpha$ y $\beta$) basándose en las variables definidas.
3.  Define las representaciones vectoriales de los estados base $|0⟩$ y $|1⟩$ utilizando arrays de NumPy.
4.  Construye el estado cuántico como una combinación lineal de los estados base utilizando las amplitudes calculadas.
5.  Imprime el estado cuántico resultante.

En el ejemplo proporcionado, con los valores específicos de las variables, las amplitudes resultantes son $c0 = 0$ y $c1 = 1$. Esto lleva a la construcción del estado:

$|\psi⟩ = 0|0⟩ + 1|1⟩ = |1⟩$

El código verifica que esta representación vectorial corresponde al estado $|1⟩$.

## Uso Práctico

Este código es un punto de partida simple para entender cómo se pueden representar los estados cuánticos en un entorno de programación clásico. Puedes modificar las variables iniciales (`x`, `y`, `z`, etc.) para ver cómo cambian las amplitudes `c0` y `c1` y, en consecuencia, cómo se representa el estado cuántico resultante.

**Para ejecutar el código:**

1.  Asegúrate de tener Python instalado.
2.  Instala la librería NumPy si aún no la tienes:

In [8]:
# Definir las variables x,y,z,n,w
x=-1
y=0.5
z=0
n=1
w=-0.5
# Definir coeficientes
c0=(x*z)
c1=(y*2)
print(f"El coeficiente c0 es: ",(c0))
print("El coeficiente c1 es: ",c1)

El coeficiente c0 es:  0
El coeficiente c1 es:  1.0


In [9]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.patches as patches
from matplotlib.colors import LinearSegmentedColormap
from qutip import Bloch
from qutip.qip.operations import snot
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap

# Configurar estilo oscuro
plt.style.use('dark_background')

# 1. Definir las variables del sistema
x = -1
y = 0.5
z = 0
n = 1
w = -0.5

# 2. Definir la "cuantización" para los coeficientes
c0 = (x * z)  # Resultado: 0
c1 = (y * 2)  # Resultado: 1.0
print(f"El coeficiente c0 calculado es: {c0}")
print(f"El coeficiente c1 calculado es: {c1}")

# 3. Definir los estados base
state_0 = np.array([[1], [0]], dtype=complex) # |0>
state_1 = np.array([[0], [1]], dtype=complex) # |1>

# 4. Construir y normalizar el estado cuántico
unnormalized_state = c0 * state_0 + c1 * state_1
norm = np.sqrt(np.abs(c0)**2 + np.abs(c1)**2)
if norm > 0:
    normalized_state = unnormalized_state / norm
else:
    normalized_state = state_1  # Estado |1> por defecto

print(f"Estado normalizado:\n{normalized_state}")

# 5. Calcular coordenadas de la esfera de Bloch
def state_to_bloch_coords(psi):
    """Convierte un estado cuántico a coordenadas de la esfera de Bloch"""
    c0, c1 = psi[0, 0], psi[1, 0]

    # Coordenadas de Bloch usando matrices de Pauli
    x = 2 * np.real(np.conj(c0) * c1)
    y = 2 * np.imag(np.conj(c0) * c1)
    z = np.abs(c0)**2 - np.abs(c1)**2

    return x, y, z

bloch_x, bloch_y, bloch_z = state_to_bloch_coords(normalized_state)

# Crear figura con subplots
fig = plt.figure(figsize=(16, 12))
fig.patch.set_facecolor('black')

# === VISUALIZACIÓN 1: ESFERA DE BLOCH ===
ax1 = fig.add_subplot(221, projection='3d')
ax1.set_facecolor('black')

# Crear esfera
u = np.linspace(0, 2 * np.pi, 100)
v = np.linspace(0, np.pi, 100)
x_sphere = np.outer(np.cos(u), np.sin(v))
y_sphere = np.outer(np.sin(u), np.sin(v))
z_sphere = np.outer(np.ones(np.size(u)), np.cos(v))

# Dibujar esfera translúcida
ax1.plot_surface(x_sphere, y_sphere, z_sphere,
                alpha=0.1, color='cyan', linewidth=0)

# Dibujar círculos de referencia
theta = np.linspace(0, 2*np.pi, 100)
# Ecuador
ax1.plot(np.cos(theta), np.sin(theta), 0, 'cyan', alpha=0.5, linewidth=1)
# Meridiano XZ
ax1.plot(np.cos(theta), 0, np.sin(theta), 'cyan', alpha=0.5, linewidth=1)
# Meridiano YZ
ax1.plot(0, np.cos(theta), np.sin(theta), 'cyan', alpha=0.5, linewidth=1)

# Ejes coordenados
ax1.quiver(0, 0, 0, 1.2, 0, 0, color='red', alpha=0.8, arrow_length_ratio=0.1)
ax1.quiver(0, 0, 0, 0, 1.2, 0, color='green', alpha=0.8, arrow_length_ratio=0.1)
ax1.quiver(0, 0, 0, 0, 0, 1.2, color='blue', alpha=0.8, arrow_length_ratio=0.1)

# Etiquetas de los ejes
ax1.text(1.3, 0, 0, 'X', color='red', fontsize=12)
ax1.text(0, 1.3, 0, 'Y', color='green', fontsize=12)
ax1.text(0, 0, 1.3, 'Z (|0⟩)', color='blue', fontsize=12)
ax1.text(0, 0, -1.3, '-Z (|1⟩)', color='blue', fontsize=12)

# Dibujar el estado cuántico como punto y vector
ax1.scatter([bloch_x], [bloch_y], [bloch_z],
           color='yellow', s=200, alpha=1, edgecolors='orange')
ax1.quiver(0, 0, 0, bloch_x, bloch_y, bloch_z,
          color='yellow', alpha=0.9, arrow_length_ratio=0.1, linewidth=3)

ax1.set_xlabel('X', color='white')
ax1.set_ylabel('Y', color='white')
ax1.set_zlabel('Z', color='white')
ax1.set_title('Esfera de Bloch\nEstado Cuántico |ψ⟩', color='white', fontsize=14)
ax1.set_xlim([-1.5, 1.5])
ax1.set_ylim([-1.5, 1.5])
ax1.set_zlim([-1.5, 1.5])

# === VISUALIZACIÓN 2: DISTRIBUCIÓN DE PROBABILIDAD ===
ax2 = fig.add_subplot(222)
ax2.set_facecolor('black')

# Probabilidades de los estados base
prob_0 = np.abs(normalized_state[0, 0])**2
prob_1 = np.abs(normalized_state[1, 0])**2

states = ['|0⟩', '|1⟩']
probabilities = [prob_0, prob_1]
colors = ['lightblue', 'lightcoral']

bars = ax2.bar(states, probabilities, color=colors, alpha=0.7, edgecolor='white')
ax2.set_ylabel('Probabilidad', color='white')
ax2.set_title('Distribución de Probabilidad\nde Estados Base', color='white', fontsize=14)
ax2.set_ylim([0, 1])
ax2.grid(True, alpha=0.3)
ax2.tick_params(colors='white')

# Añadir valores en las barras
for bar, prob in zip(bars, probabilities):
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height + 0.02,
             f'{prob:.3f}', ha='center', va='bottom', color='white', fontsize=12)

# === VISUALIZACIÓN 3: DENSIDAD DE DISTRIBUCIÓN 2D ===
ax3 = fig.add_subplot(223)
ax3.set_facecolor('black')

# Crear una cuadrícula para la distribución de densidad
x_grid = np.linspace(-2, 2, 100)
y_grid = np.linspace(-2, 2, 100)
X, Y = np.meshgrid(x_grid, y_grid)

# Función de densidad gaussiana centrada en el estado
sigma = 0.5
density = np.exp(-((X - bloch_x)**2 + (Y - bloch_y)**2) / (2 * sigma**2))

# Colormap personalizado para fondo oscuro
colors_custom = ['black', 'darkblue', 'blue', 'cyan', 'yellow', 'orange', 'red']
n_bins = 256
custom_cmap = LinearSegmentedColormap.from_list('custom', colors_custom, N=n_bins)

im = ax3.imshow(density, extent=[-2, 2, -2, 2], origin='lower',
                cmap=custom_cmap, alpha=0.8)
ax3.scatter([bloch_x], [bloch_y], color='white', s=100,
           marker='x', linewidth=3, label='Estado |ψ⟩')

ax3.set_xlabel('X', color='white')
ax3.set_ylabel('Y', color='white')
ax3.set_title('Densidad de Distribución 2D\n(Proyección XY)', color='white', fontsize=14)
ax3.legend()
ax3.grid(True, alpha=0.3)
ax3.tick_params(colors='white')

# Colorbar
cbar = plt.colorbar(im, ax=ax3)
cbar.ax.yaxis.set_tick_params(color='white')
cbar.ax.yaxis.set_ticklabels(cbar.ax.yaxis.get_ticklabels(), color='green')

# === VISUALIZACIÓN 4: COMPONENTES COMPLEJAS ===
ax4 = fig.add_subplot(224)
ax4.set_facecolor('black')

# Partes real e imaginaria de los coeficientes
real_parts = [np.real(normalized_state[0, 0]), np.real(normalized_state[1, 0])]
imag_parts = [np.imag(normalized_state[0, 0]), np.imag(normalized_state[1, 0])]

x_pos = np.arange(len(states))
width = 0.35

bars1 = ax4.bar(x_pos - width/2, real_parts, width, label='Parte Real',
                color='lightgreen', alpha=0.7, edgecolor='white')
bars2 = ax4.bar(x_pos + width/2, imag_parts, width, label='Parte Imaginaria',
                color='lightpink', alpha=0.7, edgecolor='white')

ax4.set_xlabel('Estados Base', color='white')
ax4.set_ylabel('Amplitud', color='white')
ax4.set_title('Componentes Complejas\nde los Coeficientes', color='white', fontsize=14)
ax4.set_xticks(x_pos)
ax4.set_xticklabels(states)
ax4.legend()
ax4.grid(True, alpha=0.3)
ax4.tick_params(colors='white')

# Añadir valores en las barras
for bars in [bars1, bars2]:
    for bar in bars:
        height = bar.get_height()
        ax4.text(bar.get_x() + bar.get_width()/2., height + 0.01 if height >= 0 else height - 0.05,
                f'{height:.3f}', ha='center', va='bottom' if height >= 0 else 'top',
                color='white', fontsize=10)

plt.tight_layout()
plt.show()

# Información adicional del estado
print(f"\n=== INFORMACIÓN DEL ESTADO CUÁNTICO ===")
print(f"Coordenadas de Bloch: ({bloch_x:.3f}, {bloch_y:.3f}, {bloch_z:.3f})")
print(f"Probabilidad |0⟩: {prob_0:.3f}")
print(f"Probabilidad |1⟩: {prob_1:.3f}")
print(f"Norma del estado: {np.sqrt(prob_0 + prob_1):.3f}")
print(f"Estado normalizado:\n{normalized_state}")

ImportError: Importing 'qutip.qip' requires the 'qutip_qip' package. Install it with `pip install qutip-qip` (for more details, go to https://qutip-qip.readthedocs.io/).

In [ ]:
!pip install qutip

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.patches as patches
from matplotlib.colors import LinearSegmentedColormap

# Configurar estilo oscuro
plt.style.use('dark_background')

# 1. Definir las variables del sistema (tomadas del primer script)
x = -1
y = 0.5
z = 0
n = 1
w = -0.5

# 2. Definir la "cuantización" para los coeficientes (tomada del primer script)
c0 = (x * z)
c1 = (y * 2)
print(f"El coeficiente c0 calculado es: {c0}")
print(f"El coeficiente c1 calculado es: {c1}")

# 3. Definir los estados base (tomada del segundo script)
state_0 = np.array([[1], [0]], dtype=complex) # |0>
state_1 = np.array([[0], [1]], dtype=complex) # |1>

# 4. Construir y normalizar el estado cuántico (tomada del segundo script)
unnormalized_state = c0 * state_0 + c1 * state_1
norm = np.sqrt(np.abs(c0)**2 + np.abs(c1)**2)
if norm > 0:
    normalized_state = unnormalized_state / norm
else:
    normalized_state = state_1  # Estado |1> por defecto

print(f"Estado normalizado:\n{normalized_state}")

# 5. Calcular coordenadas de la esfera de Bloch (tomada del segundo script)
def state_to_bloch_coords(psi):
    """Convierte un estado cuántico a coordenadas de la esfera de Bloch"""
    c0, c1 = psi[0, 0], psi[1, 0]

    # Coordenadas de Bloch usando matrices de Pauli
    x = 2 * np.real(np.conj(c0) * c1)
    y = 2 * np.imag(np.conj(c0) * c1)
    z = np.abs(c0)**2 - np.abs(c1)**2

    return x, y, z

bloch_x, bloch_y, bloch_z = state_to_bloch_coords(normalized_state)

# Crear figura con subplots (tomada del segundo script)
fig = plt.figure(figsize=(16, 12))
fig.patch.set_facecolor('black')

# === VISUALIZACIÓN 1: ESFERA DE BLOCH === (tomada del segundo script)
ax1 = fig.add_subplot(221, projection='3d')
ax1.set_facecolor('black')

# Crear esfera
u = np.linspace(0, 2 * np.pi, 100)
v = np.linspace(0, np.pi, 100)
x_sphere = np.outer(np.cos(u), np.sin(v))
y_sphere = np.outer(np.sin(u), np.sin(v))
z_sphere = np.outer(np.ones(np.size(u)), np.cos(v))

# Dibujar esfera translúcida
ax1.plot_surface(x_sphere, y_sphere, z_sphere,
                alpha=0.1, color='cyan', linewidth=0)

# Dibujar círculos de referencia
theta = np.linspace(0, 2*np.pi, 100)
# Ecuador
ax1.plot(np.cos(theta), np.sin(theta), 0, 'cyan', alpha=0.5, linewidth=1)
# Meridiano XZ
ax1.plot(np.cos(theta), 0, np.sin(theta), 'cyan', alpha=0.5, linewidth=1)
# Meridiano YZ
ax1.plot(0, np.cos(theta), np.sin(theta), 'cyan', alpha=0.5, linewidth=1)

# Ejes coordenados
ax1.quiver(0, 0, 0, 1.2, 0, 0, color='red', alpha=0.8, arrow_length_ratio=0.1)
ax1.quiver(0, 0, 0, 0, 1.2, 0, color='green', alpha=0.8, arrow_length_ratio=0.1)
ax1.quiver(0, 0, 0, 0, 0, 1.2, color='blue', alpha=0.8, arrow_length_ratio=0.1)

# Etiquetas de los ejes
ax1.text(1.3, 0, 0, 'X', color='red', fontsize=12)
ax1.text(0, 1.3, 0, 'Y', color='green', fontsize=12)
ax1.text(0, 0, 1.3, 'Z (|0⟩)', color='blue', fontsize=12)
ax1.text(0, 0, -1.3, '-Z (|1⟩)', color='blue', fontsize=12)

# Dibujar el estado cuántico como punto y vector
ax1.scatter([bloch_x], [bloch_y], [bloch_z],
           color='yellow', s=200, alpha=1, edgecolors='orange')
ax1.quiver(0, 0, 0, bloch_x, bloch_y, bloch_z,
          color='yellow', alpha=0.9, arrow_length_ratio=0.1, linewidth=3)

ax1.set_xlabel('X', color='white')
ax1.set_ylabel('Y', color='white')
ax1.set_zlabel('Z', color='white')
ax1.set_title('Esfera de Bloch\nEstado Cuántico |ψ⟩', color='white', fontsize=14)
ax1.set_xlim([-1.5, 1.5])
ax1.set_ylim([-1.5, 1.5])
ax1.set_zlim([-1.5, 1.5])

# === VISUALIZACIÓN 2: DISTRIBUCIÓN DE PROBABILIDAD === (tomada del segundo script)
ax2 = fig.add_subplot(222)
ax2.set_facecolor('black')

# Probabilidades de los estados base
prob_0 = np.abs(normalized_state[0, 0])**2
prob_1 = np.abs(normalized_state[1, 0])**2

states = ['|0⟩', '|1⟩']
probabilities = [prob_0, prob_1]
colors = ['lightblue', 'lightcoral']

bars = ax2.bar(states, probabilities, color=colors, alpha=0.7, edgecolor='white')
ax2.set_ylabel('Probabilidad', color='white')
ax2.set_title('Distribución de Probabilidad\nde Estados Base', color='white', fontsize=14)
ax2.set_ylim([0, 1])
ax2.grid(True, alpha=0.3)
ax2.tick_params(colors='white')

# Añadir valores en las barras
for bar, prob in zip(bars, probabilities):
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height + 0.02,
             f'{prob:.3f}', ha='center', va='bottom', color='white', fontsize=12)

# === VISUALIZACIÓN 3: DENSIDAD DE DISTRIBUCIÓN 2D === (tomada del segundo script)
ax3 = fig.add_subplot(223)
ax3.set_facecolor('black')

# Crear una cuadrícula para la distribución de densidad
x_grid = np.linspace(-2, 2, 100)
y_grid = np.linspace(-2, 2, 100)
X, Y = np.meshgrid(x_grid, y_grid)

# Función de densidad gaussiana centrada en el estado
sigma = 0.5
density = np.exp(-((X - bloch_x)**2 + (Y - bloch_y)**2) / (2 * sigma**2))

# Colormap personalizado para fondo oscuro
colors_custom = ['black', 'darkblue', 'blue', 'cyan', 'yellow', 'orange', 'red']
n_bins = 256
custom_cmap = LinearSegmentedColormap.from_list('custom', colors_custom, N=n_bins)

im = ax3.imshow(density, extent=[-2, 2, -2, 2], origin='lower',
                cmap=custom_cmap, alpha=0.8)
ax3.scatter([bloch_x], [bloch_y], color='white', s=100,
           marker='x', linewidth=3, label='Estado |ψ⟩')

ax3.set_xlabel('X', color='white')
ax3.set_ylabel('Y', color='white')
ax3.set_title('Densidad de Distribución 2D\n(Proyección XY)', color='white', fontsize=14)
ax3.legend()
ax3.grid(True, alpha=0.3)
ax3.tick_params(colors='white')

# Colorbar
cbar = plt.colorbar(im, ax=ax3)
cbar.ax.yaxis.set_tick_params(color='white')
cbar.ax.yaxis.set_ticklabels(cbar.ax.yaxis.get_ticklabels(), color='green')

# === VISUALIZACIÓN 4: COMPONENTES COMPLEJAS === (tomada del segundo script)
ax4 = fig.add_subplot(224)
ax4.set_facecolor('black')

# Partes real e imaginaria de los coeficientes
real_parts = [np.real(normalized_state[0, 0]), np.real(normalized_state[1, 0])]
imag_parts = [np.imag(normalized_state[0, 0]), np.imag(normalized_state[1, 0])]

x_pos = np.arange(len(states))
width = 0.35

bars1 = ax4.bar(x_pos - width/2, real_parts, width, label='Parte Real',
                color='lightgreen', alpha=0.7, edgecolor='white')
bars2 = ax4.bar(x_pos + width/2, imag_parts, width, label='Parte Imaginaria',
                color='lightpink', alpha=0.7, edgecolor='white')

ax4.set_xlabel('Estados Base', color='white')
ax4.set_ylabel('Amplitud', color='white')
ax4.set_title('Componentes Complejas\nde los Coeficientes', color='white', fontsize=14)
ax4.set_xticks(x_pos)
ax4.set_xticklabels(states)
ax4.legend()
ax4.grid(True, alpha=0.3)
ax4.tick_params(colors='white')

# Añadir valores en las barras
for bars in [bars1, bars2]:
    for bar in bars:
        height = bar.get_height()
        ax4.text(bar.get_x() + bar.get_width()/2., height + 0.01 if height >= 0 else height - 0.05,
                f'{height:.3f}', ha='center', va='bottom' if height >= 0 else 'top',
                color='white', fontsize=10)

plt.tight_layout()
plt.show()

# Información adicional del estado (tomada del segundo script)
print(f"\n=== INFORMACIÓN DEL ESTADO CUÁNTICO ===")
print(f"Coordenadas de Bloch: ({bloch_x:.3f}, {bloch_y:.3f}, {bloch_z:.3f})")
print(f"Probabilidad |0⟩: {prob_0:.3f}")
print(f"Probabilidad |1⟩: {prob_1:.3f}")
print(f"Norma del estado: {np.sqrt(prob_0 + prob_1):.3f}")
print(f"Estado normalizado:\n{normalized_state}")

In [ ]:
import math

def wave_function(x_values, linear_phase=0.0):
    """
    Simula una función de onda 1D con un factor de fase lineal usando solo diccionarios.

    Args:
        x_values (list): Lista de coordenadas espaciales.
        linear_phase (float): Factor de fase lineal.

    Returns:
        dict: Diccionario con las coordenadas x como claves y los valores complejos como valores.
    """
    result = {}

    for x in x_values:
        # Función base: e^(-x²)
        psi_base = math.exp(-x**2)

        # Factor de fase: e^(i * linear_phase * x) = cos(linear_phase * x) + i * sin(linear_phase * x)
        phase_real = math.cos(linear_phase * x)
        phase_imag = math.sin(linear_phase * x)

        # Multiplicación de números complejos: psi_base * (phase_real + i * phase_imag)
        real_part = psi_base * phase_real
        imag_part = psi_base * phase_imag

        # Almacenar como tupla (parte_real, parte_imaginaria)
        result[x] = (real_part, imag_part)

    return result

# Función auxiliar para trabajar con números complejos como tuplas
def complex_magnitude(complex_tuple):
    """Calcula la magnitud de un número complejo representado como tupla."""
    real, imag = complex_tuple
    return math.sqrt(real**2 + imag**2)

def complex_phase(complex_tuple):
    """Calcula la fase de un número complejo representado como tupla."""
    real, imag = complex_tuple
    return math.atan2(imag, real)

# Ejemplo de uso
if __name__ == "__main__":
    # Coordenadas espaciales
    x_coords = [-2, -1.5, -1, -0.5, 0, 0.5, 1, 1.5, 2]

    # Calcular función de onda con fase lineal
    wave_result = wave_function(x_coords, linear_phase=1.0)

    print("Función de onda calculada:")
    print("x\t\tReal\t\tImag\t\tMagnitud")
    print("-" * 50)

    for x, (real, imag) in wave_result.items():
        magnitude = complex_magnitude((real, imag))
        print(f"{x:4.1f}\t\t{real:8.4f}\t{imag:8.4f}\t{magnitude:8.4f}")

    # Ejemplo adicional: comparar con diferentes fases lineales
    print("\nComparación con diferentes fases lineales:")
    phases = [0.0, 0.5, 1.0, 2.0]

    for phase in phases:
        result = wave_function([0, 1], linear_phase=phase)
        print(f"Fase {phase}: x=0 → {result[0]}, x=1 → {result[1]}")

In [ ]:
import qutip as qt
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns

# Enhanced system parameters
num_qbits = 3
barrier_properties = {
    "graphene": {"gamma": 0.8, "width": 1.5, "topological_phase": 0.2, "decoherence": 0.02},
    "boron_nitride": {"gamma": 0.4, "width": 2.0, "topological_phase": 0.1, "decoherence": 0.05},
    "silicon_oxide": {"gamma": 0.3, "width": 2.5, "topological_phase": 0.05, "decoherence": 0.08},
    "molybdenum_disulfide": {"gamma": 0.6, "width": 1.8, "topological_phase": 0.15, "decoherence": 0.03}
}

# Enhanced entanglement state creation
def create_ghz_state(num_qubits):
    """Create GHZ entangled state |000⟩ + |111⟩"""
    zero_state = qt.tensor([qt.basis(2, 0) for _ in range(num_qubits)])
    one_state = qt.tensor([qt.basis(2, 1) for _ in range(num_qubits)])
    return (zero_state + one_state).unit()

def create_w_state(num_qubits):
    """Create W entangled state |001⟩ + |010⟩ + |100⟩"""
    states = []
    for i in range(num_qubits):
        qubits = [qt.basis(2, 0) for _ in range(num_qubits)]
        qubits[i] = qt.basis(2, 1)
        states.append(qt.tensor(qubits))
    return sum(states).unit()

def create_werner_state(num_qubits, p=0.8):
    """Create Werner state: mixture of maximally entangled and maximally mixed states"""
    ghz = create_ghz_state(num_qubits)
    rho_ghz = ghz * ghz.dag()
    rho_mixed = qt.tensor([qt.identity(2)/2 for _ in range(num_qubits)])
    return p * rho_ghz + (1-p) * rho_mixed

# Enhanced Pauli operators for multi-qubit systems
def get_pauli_operators(num_qubits):
    """Generate Pauli operators for each qubit in the system"""
    operators = {'x': [], 'y': [], 'z': []}

    for i in range(num_qubits):
        # Pauli-X operators
        pauli_x = qt.tensor([qt.identity(2) if j != i else qt.sigmax() for j in range(num_qubits)])
        operators['x'].append(pauli_x)

        # Pauli-Y operators
        pauli_y = qt.tensor([qt.identity(2) if j != i else qt.sigmay() for j in range(num_qubits)])
        operators['y'].append(pauli_y)

        # Pauli-Z operators
        pauli_z = qt.tensor([qt.identity(2) if j != i else qt.sigmaz() for j in range(num_qubits)])
        operators['z'].append(pauli_z)

    return operators

# Advanced Hamiltonian construction
def create_advanced_hamiltonian(material_props, external_field=0.1, coupling_strength=0.05):
    """Create advanced Hamiltonian with multiple physical effects"""
    gamma = material_props["gamma"]
    topological_phase = material_props["topological_phase"]

    pauli_ops = get_pauli_operators(num_qbits)

    # Standard tunneling term
    H_tunnel = sum([gamma * op for op in pauli_ops['x']])

    # External magnetic field (Zeeman effect)
    H_zeeman = external_field * sum([op for op in pauli_ops['z']])

    # Nearest-neighbor interactions (Ising-like)
    H_interaction = sum([coupling_strength * pauli_ops['z'][i] * pauli_ops['z'][(i+1)%num_qbits]
                        for i in range(num_qbits)])

    # Topological term (Dzyaloshinskii-Moriya interaction)
    H_topo = topological_phase * sum([pauli_ops['x'][i] * pauli_ops['y'][(i+1)%num_qbits] -
                                     pauli_ops['y'][i] * pauli_ops['x'][(i+1)%num_qbits]
                                     for i in range(num_qbits)])

    # Long-range dipole-dipole interactions
    H_dipole = 0.01 * sum([pauli_ops['z'][i] * pauli_ops['z'][j] / (abs(i-j) + 1)
                          for i in range(num_qbits) for j in range(i+1, num_qbits)])

    return H_tunnel + H_zeeman + H_interaction + H_topo + H_dipole

# Enhanced coherence metrics
def calculate_quantum_coherence(state):
    """Calculate multiple coherence measures"""
    if isinstance(state, qt.Qobj) and state.type == 'ket':
        rho = state * state.dag()
    else:
        rho = state

    coherence_metrics = {}

    # L1-norm coherence
    rho_diag = qt.Qobj(np.diag(np.diag(rho.full())))
    coherence_metrics['l1_norm'] = (rho - rho_diag).norm('fro')

    # Relative entropy coherence
    try:
        coherence_metrics['rel_entropy'] = qt.entropy_relative(rho, rho_diag)
    except:
        coherence_metrics['rel_entropy'] = 0

    # Trace distance coherence
    coherence_metrics['trace_distance'] = qt.tracedist(rho, rho_diag)

    return coherence_metrics

def calculate_entanglement_measures(state):
    """Calculate various entanglement measures"""
    if isinstance(state, qt.Qobj) and state.type == 'ket':
        rho = state * state.dag()
    else:
        rho = state

    entanglement_metrics = {}

    try:
        # Bipartite entanglement (for 2-qubit subsystems)
        if num_qbits >= 2:
            rho_AB = rho.ptrace([0, 1])
            entanglement_metrics['concurrence_01'] = qt.concurrence(rho_AB)

        # Tripartite negativity
        if num_qbits == 3:
            entanglement_metrics['negativity_012'] = qt.negativity(rho, [0])

    except Exception as e:
        print(f"Entanglement calculation error: {e}")
        entanglement_metrics['concurrence_01'] = 0
        entanglement_metrics['negativity_012'] = 0

    return entanglement_metrics

# Comprehensive simulation function
def run_comprehensive_simulation(material="graphene"):
    """Run comprehensive quantum tunneling simulation"""
    material_props = barrier_properties[material]

    # Time evolution parameters
    tlist = np.linspace(0, 20, 200)

    # Initial states to compare
    initial_states = {
        'GHZ': create_ghz_state(num_qbits),
        'W': create_w_state(num_qbits),
        'Werner': create_werner_state(num_qbits, p=0.7)
    }

    # Hamiltonian
    H = create_advanced_hamiltonian(material_props)
    pauli_ops = get_pauli_operators(num_qbits)

    # Observables to measure
    observables = pauli_ops['x'] + pauli_ops['y'] + pauli_ops['z']

    # Decoherence operators
    c_ops = [np.sqrt(material_props["decoherence"]) * op for op in pauli_ops['z']]

    results = {}

    for state_name, initial_state in initial_states.items():
        print(f"Simulating {state_name} state...")

        # Run simulation
        result = qt.mesolve(H, initial_state, tlist, c_ops, observables)

        # Calculate time-evolved coherence and entanglement
        coherence_evolution = []
        entanglement_evolution = []

        for state_t in result.states:
            coh_metrics = calculate_quantum_coherence(state_t)
            ent_metrics = calculate_entanglement_measures(state_t)

            coherence_evolution.append(coh_metrics['l1_norm'])
            entanglement_evolution.append(ent_metrics.get('concurrence_01', 0))

        results[state_name] = {
            'result': result,
            'coherence': coherence_evolution,
            'entanglement': entanglement_evolution
        }

    return results, tlist, material_props

# Advanced visualization functions
def create_comprehensive_plots(results, tlist, material_props, material_name):
    """Create comprehensive visualization of results"""

    # Set up the plotting style
    plt.style.use('seaborn-v0_8')
    fig = plt.figure(figsize=(20, 15))

    # 1. Coherence evolution comparison
    ax1 = plt.subplot(3, 3, 1)
    for state_name, data in results.items():
        plt.plot(tlist, data['coherence'], linewidth=2.5, label=f'{state_name} state', alpha=0.8)
    plt.xlabel('Time (a.u.)')
    plt.ylabel('L1-norm Coherence')
    plt.title(f'Coherence Evolution - {material_name}')
    plt.legend()
    plt.grid(True, alpha=0.3)

    # 2. Entanglement evolution
    ax2 = plt.subplot(3, 3, 2)
    for state_name, data in results.items():
        if len(data['entanglement']) > 0:
            plt.plot(tlist, data['entanglement'], linewidth=2.5, label=f'{state_name} state', alpha=0.8)
    plt.xlabel('Time (a.u.)')
    plt.ylabel('Concurrence')
    plt.title(f'Entanglement Evolution - {material_name}')
    plt.legend()
    plt.grid(True, alpha=0.3)

    # 3. Expectation values for GHZ state (Pauli-X)
    ax3 = plt.subplot(3, 3, 3)
    ghz_result = results['GHZ']['result']
    for i in range(num_qbits):
        plt.plot(tlist, ghz_result.expect[i], linewidth=2, label=f'⟨σₓ⟩ Qubit {i+1}')
    plt.xlabel('Time (a.u.)')
    plt.ylabel('⟨σₓ⟩')
    plt.title('Pauli-X Expectation Values (GHZ)')
    plt.legend()
    plt.grid(True, alpha=0.3)

    # 4. Phase space evolution (Bloch vectors)
    ax4 = plt.subplot(3, 3, 4)
    colors = ['red', 'blue', 'green']
    for i in range(num_qbits):
        x_vals = ghz_result.expect[i]  # σₓ
        y_vals = ghz_result.expect[i + num_qbits]  # σᵧ
        plt.plot(x_vals, y_vals, color=colors[i], alpha=0.7, linewidth=2, label=f'Qubit {i+1}')
        plt.scatter(x_vals[0], y_vals[0], color=colors[i], s=50, marker='o')  # Initial
        plt.scatter(x_vals[-1], y_vals[-1], color=colors[i], s=50, marker='s')  # Final
    plt.xlabel('⟨σₓ⟩')
    plt.ylabel('⟨σᵧ⟩')
    plt.title('Phase Space Trajectories')
    plt.legend()
    plt.grid(True, alpha=0.3)

    # 5. Material comparison
    ax5 = plt.subplot(3, 3, 5)
    materials = list(barrier_properties.keys())
    final_coherences = []

    for mat in materials:
        # Quick simulation for comparison
        H_temp = create_advanced_hamiltonian(barrier_properties[mat])
        temp_result = qt.mesolve(H_temp, create_ghz_state(num_qbits),
                                np.linspace(0, 10, 50), [], [])
        if len(temp_result.states) > 0:
            final_coh = calculate_quantum_coherence(temp_result.states[-1])['l1_norm']
        else:
            final_coh = 0
        final_coherences.append(final_coh)

    bars = plt.bar(materials, final_coherences, alpha=0.7,
                   color=['red', 'blue', 'green', 'orange'][:len(materials)])
    plt.xlabel('Material')
    plt.ylabel('Final Coherence')
    plt.title('Material Comparison')
    plt.xticks(rotation=45)

    # Highlight current material
    if material_name in materials:
        idx = materials.index(material_name)
        bars[idx].set_color('gold')
        bars[idx].set_edgecolor('black')
        bars[idx].set_linewidth(2)

    # 6. Tunneling probability vs angle
    ax6 = plt.subplot(3, 3, 6)
    angles = np.linspace(0, np.pi/2, 20)
    tunnel_probs = []

    for angle in angles:
        # Modify Hamiltonian for different angles
        modified_props = material_props.copy()
        modified_props["gamma"] *= np.cos(angle)
        H_angle = create_advanced_hamiltonian(modified_props)

        # Short simulation
        short_tlist = np.linspace(0, 5, 30)
        temp_result = qt.mesolve(H_angle, create_ghz_state(num_qbits),
                                short_tlist, [], get_pauli_operators(num_qbits)['z'])

        # Calculate transmission probability
        final_sigmaz = np.mean([temp_result.expect[i][-1] for i in range(num_qbits)])
        prob = (1 - final_sigmaz) / 2  # Convert from ⟨σz⟩ to probability
        tunnel_probs.append(prob)

    plt.plot(angles * 180/np.pi, tunnel_probs, 'go-', linewidth=2.5, markersize=6)
    plt.xlabel('Incident Angle (degrees)')
    plt.ylabel('Transmission Probability')
    plt.title('Angular Dependence')
    plt.grid(True, alpha=0.3)

    # 7. Decoherence analysis
    ax7 = plt.subplot(3, 3, 7)
    decoherence_rates = np.linspace(0, 0.1, 10)
    final_coherences_vs_noise = []

    for rate in decoherence_rates:
        temp_props = material_props.copy()
        temp_props["decoherence"] = rate

        H_temp = create_advanced_hamiltonian(temp_props)
        c_ops_temp = [np.sqrt(rate) * op for op in get_pauli_operators(num_qbits)['z']]

        temp_result = qt.mesolve(H_temp, create_ghz_state(num_qbits),
                                np.linspace(0, 10, 50), c_ops_temp, [])

        if len(temp_result.states) > 0:
            final_coh = calculate_quantum_coherence(temp_result.states[-1])['l1_norm']
        else:
            final_coh = 0
        final_coherences_vs_noise.append(final_coh)

    plt.plot(decoherence_rates, final_coherences_vs_noise, 'ro-', linewidth=2.5, markersize=6)
    plt.xlabel('Decoherence Rate')
    plt.ylabel('Final Coherence')
    plt.title('Decoherence Resilience')
    plt.grid(True, alpha=0.3)

    # 8. 3D visualization of material properties
    ax8 = plt.subplot(3, 3, 8, projection='3d')
    materials = list(barrier_properties.keys())
    x_vals = [barrier_properties[m]["gamma"] for m in materials]
    y_vals = [barrier_properties[m]["width"] for m in materials]
    z_vals = [barrier_properties[m]["topological_phase"] for m in materials]
    colors_3d = [barrier_properties[m]["decoherence"] for m in materials]

    scatter = ax8.scatter(x_vals, y_vals, z_vals, c=colors_3d, s=200, cmap='viridis', alpha=0.8)
    ax8.set_xlabel('Tunneling Coefficient (γ)')
    ax8.set_ylabel('Barrier Width (nm)')
    ax8.set_zlabel('Topological Phase')
    ax8.set_title('3D Material Properties')

    # Add material labels
    for i, mat in enumerate(materials):
        ax8.text(x_vals[i], y_vals[i], z_vals[i], f'  {mat}', fontsize=8)

    plt.colorbar(scatter, label='Decoherence Rate', shrink=0.5)

    # 9. Correlation matrix of observables
    ax9 = plt.subplot(3, 3, 9)

    # Create correlation matrix from GHZ state observables
    ghz_expectations = np.array(ghz_result.expect).T  # Transpose for time x observables
    correlation_matrix = np.corrcoef(ghz_expectations.T)

    # Create labels for observables
    obs_labels = [f'σₓ_{i+1}' for i in range(num_qbits)] + \
                [f'σᵧ_{i+1}' for i in range(num_qbits)] + \
                [f'σᵢ_{i+1}' for i in range(num_qbits)]

    # Only show first 6x6 for clarity
    if len(obs_labels) > 6:
        correlation_matrix = correlation_matrix[:6, :6]
        obs_labels = obs_labels[:6]

    im = plt.imshow(correlation_matrix, cmap='RdBu_r', vmin=-1, vmax=1)
    plt.colorbar(im, label='Correlation')
    plt.xticks(range(len(obs_labels)), obs_labels, rotation=45)
    plt.yticks(range(len(obs_labels)), obs_labels)
    plt.title('Observable Correlations')

    plt.tight_layout()
    plt.savefig(f'comprehensive_quantum_analysis_{material_name}.png', dpi=300, bbox_inches='tight')
    plt.show()

# Main execution
if __name__ == "__main__":
    print("🔬 Advanced Quantum Tunneling Simulation")
    print("=" * 50)

    # Run simulation for different materials
    for material in ["graphene", "boron_nitride"]:
        print(f"\n🧪 Analyzing {material}...")
        results, tlist, material_props = run_comprehensive_simulation(material)

        # Print summary statistics
        print(f"📊 Results for {material}:")
        for state_name, data in results.items():
            initial_coh = data['coherence'][0]
            final_coh = data['coherence'][-1]
            coherence_decay = (initial_coh - final_coh) / initial_coh * 100
            print(f"  {state_name}: Coherence decay = {coherence_decay:.1f}%")

        # Create comprehensive plots
        create_comprehensive_plots(results, tlist, material_props, material)

    print("\n✅ Simulation completed! Check the generated plots for detailed analysis.")
    print("\n📈 Key insights:")
    print("- GHZ states typically show faster decoherence but stronger initial entanglement")
    print("- Werner states provide better resilience to decoherence")
    print("- Graphene shows superior coherence preservation due to lower decoherence rates")
    print("- Topological phases contribute to non-trivial quantum correlations")